This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/demand_curve).

## Demand Curve Pipeline Tutorial

This worksheet demonstrates a Wallaroo pipeline with data preprocessing, a model, and data postprocessing.

The model is a "demand curve" that predicts the expected number of units of a product that will be sold to a customer as a function of unit price and facts about the customer. Such models can be used for price optimization or sales volume forecasting.  This is purely a "toy" demonstration, but is useful for detailing the process of working with models and pipelines.

Data preprocessing is required to create the features used by the model. Simple postprocessing prevents nonsensical estimates (e.g. negative units sold).

## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed:
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.

In [179]:
import json
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas
import numpy
import conversion
from wallaroo.object import EntityNotFoundError
import pyarrow as pa

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [180]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

Now that the Wallaroo client has been initialized, we can create the workspace and call it `demandcurveworkspace`, then set it as our current workspace.  We'll also create our pipeline so it's ready when we add our models to it.

We'll set some variables and methods to create our workspace, pipelines and models.  Note that as of the July 2022 release of Wallaroo, workspace names must be unique.  Pipelines with the same name will be created as a new version when built.

In [181]:
workspace_name = 'demandcurveworkspace'
pipeline_name = 'demandcurvepipeline'
model_name = 'demandcurvemodel'
model_file_name = './demand_curve_v1.onnx'

In [182]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [183]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

demandcurve_pipeline = get_pipeline(pipeline_name)
demandcurve_pipeline

name,demandcurvepipeline
created,2023-09-01 19:35:46.308437+00:00
last_updated,2023-09-06 19:27:27.106897+00:00
deployed,False
tags,
versions,"78bc4874-1ee4-4d93-b6e5-76baa446f8bb, 2ea4da00-7521-4ffe-9e74-7b41a440021a, 0304ba8d-d1ff-47e6-acfc-b68454d09126, e4607e83-d2e1-4809-9047-a5f5d5d27468, 4d0da913-59fb-4e99-9d6b-1c18377ab34c, 9d5076cb-5752-43ab-af4f-4c904d06ba52, d06a9550-d693-445a-a6cd-b55635018c57, c95ee014-f431-4d8d-ab99-669fde88173d, bd0c6894-63bc-4cb9-b378-390640307a36, 4136fa8b-270e-49d6-836e-75f7c9c73def, 00a4fb64-302d-4a27-90ac-86e08fb6afe8, 94f0129f-f9c1-474a-89fa-7110c81c9a22, 701a759d-6d45-4d61-bfd0-b878d9a09fc3, 4a85b06f-9051-48ff-9f9e-6b6e4e4b8d68, 698c13e2-4120-4132-b72b-99cd8c0fdd99, 1314a1a4-eabc-437d-bf3f-83f54686970e"
steps,demandcurvemodel
published,False


With our workspace established, we'll upload two models:

* `demand_curve_v1.onnx`: Our demand_curve model.  We'll store the upload configuration into `demand_curve_model`.
* `postprocess.py`: A Python step that will zero out any negative values and return the output variable as "prediction".

Note that the order we upload our models isn't important - we'll be establishing the actual process of moving data from one model to the next when we set up our pipeline.

In [184]:
demand_curve_model = wl.upload_model(model_name, model_file_name, framework=wallaroo.framework.Framework.ONNX)

preprocess_input_schema = pa.schema([
    pa.field('Date', pa.string()),
    pa.field('cust_known', pa.bool_()),
    pa.field('StockCode', pa.int32()),
    pa.field('UnitPrice', pa.float32()),
    pa.field('UnitsSold', pa.int32())
])

preprocess_input_output = pa.schema([
    pa.field('tensor', pa.list_(pa.float64()))
])

preprocess_step = (wl.upload_model('preprocess', 
                        './preprocess.py', 
                        framework=wallaroo.framework.Framework.PYTHON)
                        .configure(
                            'python', 
                            input_schema=preprocess_input_schema, 
                            output_schema=preprocess_input_output
                        )
        )

input_schema = pa.schema([
    pa.field('variable', pa.list_(pa.float64()))
])


output_schema = pa.schema([
    pa.field('prediction', pa.list_(pa.float64()))
])


step = (wl.upload_model('postprocess', 
                        './postprocess.py', 
                        framework=wallaroo.framework.Framework.PYTHON)
                        .configure(
                            'python', 
                            input_schema=input_schema, 
                            output_schema=output_schema
                        )
        )

With our models uploaded, we're going to create our own pipeline and give it three steps:

* First, we apply the data to our `demand_curve_model`.
* And finally, we prepare our data for output with the `module_post`.

In [185]:
# now make a pipeline
demandcurve_pipeline.clear()
demandcurve_pipeline.add_model_step(preprocess_step)
demandcurve_pipeline.add_model_step(demand_curve_model)
demandcurve_pipeline.add_model_step(step)

name,demandcurvepipeline
created,2023-09-01 19:35:46.308437+00:00
last_updated,2023-09-06 19:27:27.106897+00:00
deployed,False
tags,
versions,"78bc4874-1ee4-4d93-b6e5-76baa446f8bb, 2ea4da00-7521-4ffe-9e74-7b41a440021a, 0304ba8d-d1ff-47e6-acfc-b68454d09126, e4607e83-d2e1-4809-9047-a5f5d5d27468, 4d0da913-59fb-4e99-9d6b-1c18377ab34c, 9d5076cb-5752-43ab-af4f-4c904d06ba52, d06a9550-d693-445a-a6cd-b55635018c57, c95ee014-f431-4d8d-ab99-669fde88173d, bd0c6894-63bc-4cb9-b378-390640307a36, 4136fa8b-270e-49d6-836e-75f7c9c73def, 00a4fb64-302d-4a27-90ac-86e08fb6afe8, 94f0129f-f9c1-474a-89fa-7110c81c9a22, 701a759d-6d45-4d61-bfd0-b878d9a09fc3, 4a85b06f-9051-48ff-9f9e-6b6e4e4b8d68, 698c13e2-4120-4132-b72b-99cd8c0fdd99, 1314a1a4-eabc-437d-bf3f-83f54686970e"
steps,demandcurvemodel
published,False


And with that - let's deploy our model pipeline.  This usually takes about 45 seconds for the deployment to finish.

In [186]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("1Gi").build()
demandcurve_pipeline.deploy(deployment_config=deploy_config)

name,demandcurvepipeline
created,2023-09-01 19:35:46.308437+00:00
last_updated,2023-09-06 19:29:49.915077+00:00
deployed,True
tags,
versions,"bfbea1cf-0f56-46f8-9def-4fa8a4dcb0a6, 78bc4874-1ee4-4d93-b6e5-76baa446f8bb, 2ea4da00-7521-4ffe-9e74-7b41a440021a, 0304ba8d-d1ff-47e6-acfc-b68454d09126, e4607e83-d2e1-4809-9047-a5f5d5d27468, 4d0da913-59fb-4e99-9d6b-1c18377ab34c, 9d5076cb-5752-43ab-af4f-4c904d06ba52, d06a9550-d693-445a-a6cd-b55635018c57, c95ee014-f431-4d8d-ab99-669fde88173d, bd0c6894-63bc-4cb9-b378-390640307a36, 4136fa8b-270e-49d6-836e-75f7c9c73def, 00a4fb64-302d-4a27-90ac-86e08fb6afe8, 94f0129f-f9c1-474a-89fa-7110c81c9a22, 701a759d-6d45-4d61-bfd0-b878d9a09fc3, 4a85b06f-9051-48ff-9f9e-6b6e4e4b8d68, 698c13e2-4120-4132-b72b-99cd8c0fdd99, 1314a1a4-eabc-437d-bf3f-83f54686970e"
steps,demandcurvemodel
published,False


We can check the status of our pipeline to make sure everything was set up correctly:

In [187]:
demandcurve_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.3.140',
   'name': 'engine-d4fccd95c-cw5q7',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'demandcurvepipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'postprocess',
      'version': 'ec9a3eea-2ada-4cf0-b9a7-bd5bc5eb927a',
      'sha': 'ecf1a555bb27bcf62bfa42922cf69db23e7188f456015fe8299f02867c3167b2',
      'status': 'Running'},
     {'name': 'demandcurvemodel',
      'version': '02fd5dc1-cf10-4b09-bf9d-191ec19bbb5e',
      'sha': '2820b42c9e778ae259918315f25afc8685ecab9967bad0a3d241e6191b414a0d',
      'status': 'Running'},
     {'name': 'preprocess',
      'version': '0bcb667e-cac7-4f45-884c-56171667dcc4',
      'sha': '3b2b32fb6408ccee5a1886ef5cdb493692080ff6699f49de518b20d9a6f6a133',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.4.163',
   'name': 'engine-lb-584f54c899-9pdbj',
   'status': 'Running',
   '

Everything is ready.  Let's feed our pipeline some data.  We have some information prepared with the `daily_purchasses.csv` spreadsheet.  We'll start with just one row to make sure that everything is working correctly.

In [188]:
# read in some purchase data
purchases = pandas.read_csv('daily_purchases.csv')

# start with a one-row data frame for testing
subsamp_raw = purchases.iloc[0:1,: ]
subsamp_raw

,Date,cust_known,StockCode,UnitPrice,UnitsSold
0,2010-12-01,False,21928,4.21,1


In [189]:
result = demandcurve_pipeline.infer(subsamp_raw)
display(result)

,time,in.Date,in.StockCode,in.UnitPrice,in.UnitsSold,in.cust_known,out.prediction,check_failures
0,2023-09-06 19:30:05.985,2010-12-01,21928,4.21,1,False,[6.68025518653071],0


We can see from the `out.prediction` field that the demand curve has a predicted slope of 6.68 from our sample data.  We can isolate that by specifying just the data output below.

In [190]:
display(result.loc[0, ['out.prediction']][0])

[6.68025518653071]

# Bulk Inference

The initial test went perfectly.  Now let's throw some more data into our pipeline.  We'll draw 10 random rows from our spreadsheet, perform an inference from that, and then display the results and the logs showing the pipeline's actions.

In [191]:
ix = numpy.random.choice(purchases.shape[0], size=10, replace=False)
converted = conversion.pandas_to_dict(purchases.iloc[ix,: ])
test_df = pd.DataFrame(converted['query'], columns=converted['colnames'])
display(test_df)

output = demandcurve_pipeline.infer(test_df)
display(output)

,Date,cust_known,StockCode,UnitPrice,UnitsSold
0,2011-12-07,False,22411,4.13,10
1,2011-07-20,True,23203,1.79,100
2,2011-06-30,False,23200,4.13,1
3,2011-06-28,True,85099B,2.08,13
4,2011-10-09,True,85099B,2.08,22
5,2011-02-16,True,85099B,1.95,30
6,2011-09-02,False,21930,4.13,4
7,2011-01-14,False,85099F,4.13,1
8,2011-11-07,True,22385,2.08,21
9,2011-11-25,True,85099F,2.08,10


,time,in.Date,in.StockCode,in.UnitPrice,in.UnitsSold,in.cust_known,out.prediction,check_failures
0,2023-09-06 19:30:06.440,2011-12-07,22411,4.13,10,False,[6.771545926800889],0
1,2023-09-06 19:30:06.440,2011-07-20,23203,1.79,100,True,[49.73419363867448],0
2,2023-09-06 19:30:06.440,2011-06-30,23200,4.13,1,False,[6.771545926800889],0
3,2023-09-06 19:30:06.440,2011-06-28,85099B,2.08,13,True,[33.125323160373426],0
4,2023-09-06 19:30:06.440,2011-10-09,85099B,2.08,22,True,[33.125323160373426],0
5,2023-09-06 19:30:06.440,2011-02-16,85099B,1.95,30,True,[40.57067889202562],0
6,2023-09-06 19:30:06.440,2011-09-02,21930,4.13,4,False,[6.771545926800889],0
7,2023-09-06 19:30:06.440,2011-01-14,85099F,4.13,1,False,[6.771545926800889],0
8,2023-09-06 19:30:06.440,2011-11-07,22385,2.08,21,True,[33.125323160373426],0
9,2023-09-06 19:30:06.440,2011-11-25,85099F,2.08,10,True,[33.125323160373426],0


## Undeploy the Pipeline

Once we've finished with our demand curve demo, we'll undeploy the pipeline and give the resources back to our Kubernetes cluster.

In [192]:
demandcurve_pipeline.undeploy()

name,demandcurvepipeline
created,2023-09-01 19:35:46.308437+00:00
last_updated,2023-09-06 19:29:49.915077+00:00
deployed,False
tags,
versions,"bfbea1cf-0f56-46f8-9def-4fa8a4dcb0a6, 78bc4874-1ee4-4d93-b6e5-76baa446f8bb, 2ea4da00-7521-4ffe-9e74-7b41a440021a, 0304ba8d-d1ff-47e6-acfc-b68454d09126, e4607e83-d2e1-4809-9047-a5f5d5d27468, 4d0da913-59fb-4e99-9d6b-1c18377ab34c, 9d5076cb-5752-43ab-af4f-4c904d06ba52, d06a9550-d693-445a-a6cd-b55635018c57, c95ee014-f431-4d8d-ab99-669fde88173d, bd0c6894-63bc-4cb9-b378-390640307a36, 4136fa8b-270e-49d6-836e-75f7c9c73def, 00a4fb64-302d-4a27-90ac-86e08fb6afe8, 94f0129f-f9c1-474a-89fa-7110c81c9a22, 701a759d-6d45-4d61-bfd0-b878d9a09fc3, 4a85b06f-9051-48ff-9f9e-6b6e4e4b8d68, 698c13e2-4120-4132-b72b-99cd8c0fdd99, 1314a1a4-eabc-437d-bf3f-83f54686970e"
steps,demandcurvemodel
published,False


Thank you for being a part of this demonstration.  If you have additional questions, please feel free to contact us at Wallaroo.